In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-07-01 20:31:23,646 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [34]:
kwargs = {
    "game": "iRacing",
    # "track": "oschersleben gp",
    # "track": "aragon gp",
    "track": "jerez gp",
    "car": "Ferrari 488 GT3 Evo 2020",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
# laps = list(laps)
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-07-01 20:45:37,187 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-01 20:45:37,188 INFO   track.id 61 car.id 9
2023-07-01 20:45:37,188 INFO   session 1688223703 lap.id 173644 number 3
2023-07-01 20:45:37,188 INFO   length 4369 time 102.7718 valid True
2023-07-01 20:45:37,189 INFO   start 2023-07-01 17:05:13.753633+00:00 end 2023-07-01 17:06:56.538103+00:00
2023-07-01 20:45:38,936 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-01 20:45:38,937 INFO   track.id 61 car.id 9
2023-07-01 20:45:38,937 INFO   session 1688234529 lap.id 174296 number 2
2023-07-01 20:45:38,937 INFO   length 4369 time 102.958 valid True
2023-07-01 20:45:38,937 INFO   start 2023-07-01 20:03:56.184552+00:00 end 2023-07-01 20:05:39.155421+00:00
2023-07-01 20:45:38,945 ERROR No data found for 1688234529 lap 2
2023-07-01 20:45:38,946 INFO No data found for lap in fast_laps bucket, trying in default bucket
2023-07-01 20:45:38,946 INFO Fetching telem

In [54]:
# influx_data_frames = []
# for df in data_frames:
#     influx_data_frames.append(df.copy())

# data_frames = []
# for df in influx_data_frames:
#     data_frames.append(df.copy())

In [55]:
current_sectors = fast_lap_analyzer.current_fast_lap_sectors()
display(current_sectors)

[{'start': 3284, 'end': 355},
 {'start': 356, 'end': 685},
 {'start': 686, 'end': 1003},
 {'start': 1004, 'end': 1604},
 {'start': 1605, 'end': 3283}]

In [56]:
fig = lap_fig(data_frames[0], full_range=True, columns=["Throttle"])
fig.show()

In [57]:
# for i, df in enumerate(data_frames):
#     fig = lap_fig(df, full_range=True, columns=["Throttle"])
#     fig.show()
sector_start_end, df_max = fast_lap_analyzer.extract_sectors(data_frames)
display(sector_start_end)

# are the sectors similar to the current fast lap?
similarity = fast_lap_analyzer.similar_sectors(sector_start_end, current_sectors)
display(similarity)

fig = lap_fig(df_max, full_range=True, columns=["Throttle"])
# add a vertical line for each sector
for i, sector in enumerate(sector_start_end):
    fig_add_shape(fig, x0=sector["start"], x1=sector["start"], color="blue")
    fig_add_shape(fig, x0=sector["end"], x1=sector["end"], color="red")
fig.show()

2023-07-01 21:03:03,505 DEBUG Lap too short, skipping


[{'start': 4243, 'end': 355, 'length': 481},
 {'start': 356, 'end': 685, 'length': 329},
 {'start': 686, 'end': 999, 'length': 313},
 {'start': 1000, 'end': 1612, 'length': 612},
 {'start': 1613, 'end': 2277, 'length': 664},
 {'start': 2278, 'end': 2744, 'length': 466},
 {'start': 2745, 'end': 3132, 'length': 387},
 {'start': 3133, 'end': 3609, 'length': 476},
 {'start': 3610, 'end': 4242, 'length': 632}]

False

In [58]:
segments, used_laps = fast_lap_analyzer.extract_segments(sector_start_end, data_frames, laps_with_telemetry, df_max)
for i in range(len(segments)):
    segment = segments[i]
    fig = lap_fig(segment.telemetry_for_fig(), columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        fig_add_features(fig, segment.brake_features())
    throttle_features = segment.throttle_features()
    if throttle_features:
        fig_add_features(fig, segment.throttle_features(), color="green")
    fig.show()

2023-07-01 21:03:14,523 ERROR sector is empty: 6
2023-07-01 21:03:14,547 ERROR sector is empty: 6
2023-07-01 21:03:14,573 ERROR sector is empty: 6
2023-07-01 21:03:14,599 ERROR sector is empty: 6
